<a href="https://colab.research.google.com/github/kimjaehwankimjaehwan/python_/blob/main/Hugging_Face%EC%9D%98_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 이 예제에서는 Hugging Face의 Transformers 라이브러리를 사용하고, 텍스트 분류 작업을 수행하겠습니다. 내장 데이터셋으로는 GLUE의 SST-2(Stanford Sentiment Treebank) 데이터셋을 사용하였습니다.

In [1]:
!pip install transformers datasets torch

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# 데이터셋 로드
dataset = load_dataset("glue", "sst2")

# 최신 트랜스포머 모델 로드 (여기서는 RoBERTa-large를 사용)
model_name = "roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 데이터 전처리 함수
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True, padding="max_length")

# 데이터셋 전처리
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# 학습 인자 설정
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

# Trainer 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

# 모델 학습
trainer.train()

# 모델 평가
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

# 테스트 예측
test_sentence = "This movie was fantastic!"
inputs = tokenizer(test_sentence, return_tensors="pt")
outputs = model(**inputs)
predicted_class = torch.argmax(outputs.logits).item()
print(f"Predicted sentiment for '{test_sentence}': {'Positive' if predicted_class == 1 else 'Negative'}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

Step,Training Loss
10,0.694300
20,0.696500
30,0.673700
40,0.689200
50,0.703100
60,0.696800
70,0.693600
80,0.688700


Step,Training Loss
10,0.694300
20,0.696500
30,0.673700
40,0.689200
50,0.703100
60,0.696800
70,0.693600
80,0.688700


이 코드는 다음과 같은 작업을 수행합니다:

GLUE의 SST-2 데이터셋을 로드합니다.

RoBERTa-large 모델과 토크나이저를 로드합니다.

데이터를 전처리하고 토큰화합니다.

모델을 학습시킵니다.

학습된 모델을 평가합니다.

테스트 문장에 대해 감성 분석을 수행합니다.

이 코드는 최신 트랜스포머 모델인 RoBERTa-large를 사용하여 감성 분석 작업을 수행합니다.

필요에 따라 다른 모델이나 작업으로 쉽게 변경할 수 있습니다.